# download


    !curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt --output input.txt

    !wget -O input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [5]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    print(len(text))

1115394


In [6]:
# unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars), vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [7]:
# tokenizer - typically subword level:
# tiktoken bpe from openai used in gpts, and sentencepiece from google, for llama, mistral
# many possibilities for tokenization, different schema
# trade off code book vocabulary size and encoded sequence length:
# small vocabularies and long sequence length, or
# large vocabularies and short sequence length 

# !pip install tiktoken
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
enc = tiktoken.encoding_for_model("gpt-4o")
assert enc.decode(enc.encode("hello world")) == "hello world"
print(enc.encode('hello world '))

# !pip install sentencepiece
# !pip install protobuf
# https://github.com/google/sentencepiece
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='spm.model)
print(s.encode('hello world'))

SyntaxError: unterminated string literal (detected at line 19) (2160374157.py, line 19)

In [ ]:

# create simple mapping between integer and chars
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s :  [stoi[c] for c in s] # encode string to list of ints
decode = lambda l :  ''.join([itos[i] for i in l]) # decode list of ints to string

print(encode("hello how are you"))
print(decode(encode("hello how are you")))

[46, 43, 50, 50, 53, 1, 46, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59]
hello how are you


In [ ]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:10])
print(decode(encode(text[:20])))

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])
First Citizen:
Befor


In [ ]:
# train test split
n = int(len(data) * 0.9) # first 90% will be train
train_data = data[:n]
val_data = data[n:]
print(len(train_data), len(val_data))

1003854 111540


In [ ]:
# we work only with chunk of token dataset
# with additional parameter block_size
# which is longest context 

block_size = 8
x = train_data[:block_size] # take a trunk
y = train_data[1:block_size+1] # take the moving y

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
# add batch function to allow batch processing

torch.manual_seed(1337)
batch_size = 4 # how many sequences with be processed in parallel
block_size = 8 # what is the matximum context length for prediction

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return  x, y

xb, yb = get_batch('train')

print('inputs', xb.shape, yb.shape)
print(xb)
print(yb)

print('-------')
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

## ?? 
#  this may make training less effective, as always same t is used in one step
#  if t is different among the batch, may be good
#  or gradient accumulation with multiple steps, from different t 


inputs torch.Size([4, 8]) torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1

In [ ]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [ ]:
# implement bigram language model

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits
        # for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):

        # idx and targes are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # B, T, C [4, 8, 65]

        if targets == None:
            loss = None
        else:
            # reshape to match torch ce loss input format
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)        
            loss = F.cross_entropy(logits, targets) 
        
        return logits, loss
    
    def generate(self, idx, max_new_token):
        # idx is (B, T) array of indices in current context
        for _ in range(max_new_token):
            # get the prediction
            logits, loss = self(idx) # by default call forward
            # focus only on the last time step
            logits = logits[:,-1,:] # the last T tells next token, becomes (B, C)
            # apply softmax to get probs
            probs = F.softmax(logits, dim = -1) # norm along channel, (B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx


    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

print(vocab_size)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print('sample idx:', idx)
print('random model:', decode(m.generate(idx, max_new_token=100)[0].tolist()))


65
tensor(4.6691, grad_fn=<NllLossBackward0>)
sample idx: tensor([[0]])
random model: 
!zXNncOx;Y-ua,iopHr?CBpD:OOEfUVXEEErpQWolyntZoM-AH,uIXvV;GduRO
R-,yU3KRNAatvLDE-ib3.quhv.hOL.R3hCSG.


In [ ]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)
batch_size = 64
for steps in range(100000):
    
    # sample a batch
    xb, yb = get_batch('train')

    # get loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True) # set to none = True faster
    loss.backward()
    optimizer.step()
    if steps%10000 == 0:
        print(loss.item()) 

print(decode(m.generate(idx=torch.zeros((1,1), dtype = torch.long),
                        max_new_token=300
                        )[0].tolist()))

2.494992256164551
2.4298572540283203
2.418480634689331
2.441776752471924
2.42606520652771
2.5111498832702637
2.5172455310821533
2.4108800888061523
2.4815258979797363


KeyboardInterrupt: 

In [81]:
# bigram limits the interaction: T+1 token is soly predicted based on the logits at time step T
# straight foward thought: merge more histories, e.g t from 1 to T-1

# The mathematical trick in self-attention
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1337)
B,T,C = 4, 8, 2 # batch, time and channel
x = torch.randn(B,T,C)

# naive way: average all logits from 1 to t
# cons: too much information compression


In [82]:
# pytorch is efficient in matrix multiplication
# by manipulating the elements of matrix a
a = torch.ones(3,3)
a = torch.tril(a) # get lower triangular portion of matrix
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(low=0, high=10, size=(3,2)).float()
c = a @ b # matmul with 
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[8., 6.],
        [5., 2.],
        [4., 4.]])
tensor([[8.0000, 6.0000],
        [6.5000, 4.0000],
        [5.6667, 4.0000]])


In [104]:
# implementations
# we want x[b,t] = mean_{i<=t}(x[b, i])

# version 1: two loops
xbow = torch.zeros(B, T, C) # bow: bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b, t] = torch.mean(xprev, 0) # collapse axis 0

# version 2: matmul
wei = torch.tril(torch.ones(T, T)) # weights for weighted sum, how much info to integrate
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # [B, T, T] @ [B, T, C] => [B, T, C]
print(torch.max(xbow-xbow2), torch.min(xbow-xbow2))

# version 3: softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T) 
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1) # change all 1 to exp(1), does not change weight
xbow3 = wei @ x

print(torch.max(xbow2-xbow3), torch.min(xbow2-xbow3))
print(torch.allclose(xbow2, xbow3))

# version 4: self attention
torch.manual_seed(1337)
B,T,C=4,8,32 # batch, time, channel
x = torch.randn(B, T, C)

#---------
# insert single head self-attention here to allow communication with history 
head_size = 16
sqrt_k = head_size**-0.5 # control softmax output, peaky or not
query = nn.Linear(C, head_size, bias = False) ## ?? make query2 = query - key ??
key = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False) 

# q and k makes weights: wei
q = query(x) # [B, T, head_size]
k = key(x) # [B, T, head_size]

# v replaces x
v = value(x) # [B, T, head_size]

# create weight matrix with temporal interactions
wei = q @ k.transpose(-2,-1) # [B,T,16] @ [B, 16, T] ==> [B,T,T]
tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0, float('-inf'))  / sqrt_k
wei = F.softmax(wei , dim=-1)
out = wei @ v 
# --------


print(out.shape) # [B, T, head_size]
print(wei.shape) # [B, T, T]
print(wei[0])

tensor(1.1921e-07) tensor(-1.1921e-07)
tensor(0.) tensor(0.)
True
torch.Size([4, 8, 16])
torch.Size([4, 8, 8])
tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.0503e-01, 1.9497e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [7.6948e-01, 1.2167e-01, 1.0885e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [7.6411e-03, 9.9236e-01, 2.9230e-07, 4.4893e-08, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [4.0576e-06, 4.6896e-08, 5.6574e-06, 3.0854e-05, 9.9996e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.6398e-01, 2.4148e-02, 1.2273e-06, 3.4744e-06, 1.1864e-02, 1.5523e-08,
         0.0000e+00, 0.0000e+00],
        [1.9922e-01, 2.1498e-03, 3.6252e-03, 1.5262e-05, 1.1890e-02, 7.8280e-01,
         2.9956e-04, 0.0000e+00],
        [7.6736e-04, 1.5149e-02, 1.0315e-04, 7.7544e-05, 1.9058e-01, 6.1738e-01,
   

In [98]:
wei[0]

tensor([[-1.7629,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-3.3334, -1.6556,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-1.0226, -1.2606,  0.0762,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.7836, -0.8014, -0.3368, -0.8496,    -inf,    -inf,    -inf,    -inf],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,    -inf,    -inf,    -inf],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,    -inf,    -inf],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,    -inf],
        [-1.8044, -0.4126, -0.8306,  0.5899, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)